<a href="https://colab.research.google.com/github/shawn-kg/DataMining/blob/main/assign7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RNN

##Data Processing

In [1]:
import numpy as np
f = open("./Train_Arabic_Digit.txt")
f.readline()

Train_Data = dict()

m = []
count = 0
count2 = 0
for line in f:
  if (line.isspace()):
    Train_Data[count] = np.array(m.copy())
    count+=1
    m = []
    continue
  count2+=1
  m.append([float(num) for num in line.split()])
f.close()

f2 = open("./Test_Arabic_Digit.txt")
f2.readline()
Test_Data = dict()
m2 = []
count = 0
for line in f2:
  if (line.isspace()):
    Test_Data[count] = np.array(m2.copy())
    count+=1
    m2 = []
    continue
  m2.append([float(num) for num in line.split()])
f2.close()

print(len(Train_Data))
print(len(Test_Data))
print(count2)

6599
2199
263256


In [2]:
target = np.zeros((10,10))
for m in range(10):
  target[m][m] = 1
print(target)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


##RNN Algorithm

In [7]:
import scipy.special
np.random.seed(42)
numneur = 32
b_h = np.random.random_sample((numneur,1))
b_o = np.random.random_sample((10,1))
W_i = np.random.random_sample((13,numneur))
W_h = np.random.random_sample((numneur,numneur))
W_o = np.random.random_sample((numneur,10))

order = np.array(range(len(Train_Data)))

error = 0

for z in range(20):
  np.random.shuffle(order)
  acc_count = 0
  error = 0
  for i in order:
    tau = len(Train_Data[i]);
    h = np.zeros((tau+1,numneur))


    # forward propagation
    h_0 = np.zeros((numneur))
    h[0] = h_0
    for t in range(1,tau):
      h_prev = h[t-1].reshape(-1,1)
      netzj = np.dot(W_i.transpose(),Train_Data[i][t].reshape(-1,1)) + np.dot(W_h.transpose(),h_prev) + b_h
      h_t = np.zeros((numneur,1))
      # relu
      for j in range(numneur):
        if (netzj[j]<=0):
          h_t[j] = 0
        else:
          h_t[j] = netzj[j]
      h[t] = h_t.reshape(1,-1)
    netoi = np.dot(W_o.transpose(),h[-1].reshape(-1,1)) + b_o
    Oi = scipy.special.softmax(netoi)

    ind = i // 660
    pred_ind = np.argmax(Oi)
    if ind==pred_ind:
      acc_count+=1
    CE = 0
    for l in range(10):
      CE += target[ind][l]*np.log(Oi[l])
    CE = -CE
    error+=CE


    # back propagation
    So = Oi - target[ind].reshape(-1,1)
    reluDer = np.zeros((numneur))
    for d in range(numneur):
      if h[-1][d]<=0:
        reluDer[d] = 0
      else:
        reluDer[d] = 1
    temp = W_o@So
    reluDer = reluDer.reshape(-1,1)
    Sh = np.multiply(reluDer, temp)
    S = np.zeros((tau,numneur))
    S[-1] = Sh.reshape(1,-1)
    for t in reversed(range(tau-1)):
      reluDeritr = np.zeros((numneur))
      for d in range(numneur):
        if h[t][d]<=0:
          reluDeritr[d] = 0
        else:
          reluDeritr[d] = 1
      tempitr = W_h@S[t+1]
      reluDeritr = reluDeritr.reshape(-1,1)
      Sh_tr = np.multiply(reluDeritr, tempitr.reshape(-1,1))
      S[t] = Sh_tr.reshape(1,-1)
    gradb_o = So
    gradw_o = np.dot(h[-1].reshape(-1,1),So.transpose())
    gradb_h = 0
    for t in range(tau):
      gradb_h+=S[t]
    gradw_h = np.zeros(W_h.shape)
    for m in range(1,tau):
      gradw_h+=h[m-1]@S[m].transpose()
    gradw_i = 0
    for t in range(tau):
      gradw_i += np.dot(Train_Data[i][t].reshape(-1,1),S[t].reshape(-1,1).transpose())
    b_o = b_o - .001*gradb_o
    W_o = W_o - .001*gradw_o
    b_h = b_h - .001*gradb_h.reshape(-1,1)
    W_h = W_h - .001*gradw_h
    W_i = W_i - .001*gradw_i
  # print("Output Bias and Weight gradients")
  # print(gradb_o)
  # print(gradw_o)
  # print("Hidden Bias and Weight gradients")
  # print(gradb_h)
  # print(gradw_h)
  # print("Input Weight gradient")
  # print(gradw_i)
  print(acc_count/6559)
  print(error/6559)


0.09727092544595213
[2.34711684]
0.0997103216953804
[2.32502387]
0.0974233877115414
[2.31912468]
0.09193474615032779
[2.31763389]
0.09864308583625553
[2.31719238]
0.09483152919652386
[2.31708692]
0.09345936880622047
[2.31705817]
0.09528891599329166
[2.31705965]
0.08964781216648879
[2.31703378]
0.09391675560298826
[2.31704116]
0.09650861411800579
[2.3170267]
0.09544137825888092
[2.31704512]
0.0901051989632566
[2.31703759]
0.09345936880622047
[2.31701569]
0.09696600091477359
[2.31703814]
0.09681353864918432
[2.31703375]
0.09422168013416679
[2.31704296]
0.09437414239975606
[2.31699597]
0.09483152919652386
[2.31702717]
0.09361183107180973
[2.31704203]


I could not get my accuracy over 10%.

In [11]:
count=0
for i in order:
  tau = len(Train_Data[i]);
  h = np.zeros((tau+1,numneur))


  # forward propagation
  h_0 = np.zeros((numneur))
  h[0] = h_0
  for t in range(1,tau):
    h_prev = h[t-1].reshape(-1,1)
    netzj = np.dot(W_i.transpose(),Train_Data[i][t].reshape(-1,1)) + np.dot(W_h.transpose(),h_prev) + b_h
    h_t = np.zeros((numneur,1))
    # relu
    for j in range(numneur):
      if (netzj[j]<=0):
        h_t[j] = 0
      else:
        h_t[j] = netzj[j]
    h[t] = h_t.reshape(1,-1)
  netoi = np.dot(W_o.transpose(),h[-1].reshape(-1,1)) + b_o
  Oi = scipy.special.softmax(netoi)
  
  CE = 0
  for l in range(10):
    CE += target[ind][l]*np.log(Oi[l])
  CE = -CE
  error+=CE
  ind = i // 660
  pred_ind = np.argmax(Oi)
  if ind==pred_ind:
    count+=1
print("Accuracy")
print(count/6559)
print("Cross Entropy Loss")
print(error/6559)

Accuracy
0.100625095288916
Cross Entropy Loss
[8.49403276]


In [12]:
count = 0
for z in Test_Data.keys():
  tau = len(Test_Data[z]);
  h = np.zeros((tau+1,numneur))


  # forward propagation
  h_0 = np.zeros((numneur))
  h[0] = h_0
  for t in range(1,tau):
    h_prev = h[t-1].reshape(-1,1)
    netzj = np.dot(W_i.transpose(),Test_Data[z][t].reshape(-1,1)) + np.dot(W_h.transpose(),h_prev) + b_h
    h_t = np.zeros((numneur,1))
    # relu
    for j in range(numneur):
      if (netzj[j]<=0):
        h_t[j] = 0
      else:
        h_t[j] = netzj[j]
    h[t] = h_t.reshape(1,-1)
  netoi = np.dot(W_o.transpose(),h[-1].reshape(-1,1)) + b_o
  Oi = scipy.special.softmax(netoi)
  CE = 0
  for l in range(10):
    CE += target[ind][l]*np.log(Oi[l])
  CE = -CE
  error+=CE

  ind = z // 220
  pred_ind = np.argmax(Oi)
  if ind==pred_ind:
    count+=1
print("Accuracy")
print(count/2199)
print("Cross Entropy Loss")
print(error/2199)

Accuracy
0.10004547521600728
Cross Entropy Loss
[27.63790337]
